In [1]:
import requests
import http.client
import pandas as pd
import json

In [65]:
ticker_dict={'Technology':['MSFT', 'AAPL', 'AMZN', 'GOOGL','META'],
'Financial':['JPM', 'BAC', 'WFC', 'GS','C'],
'Healthcare':['JNJ', 'PFE', 'MRK', 'UNH', 'AMGN'],
'Consumer Discretionary':['NFLX', 'DIS', 'TSLA', 'NKE', 'MCD'],
'Consumer Staples':['PG', 'KO', 'PEP', 'WMT', 'COST'],
'Energy':['XOM', 'CVX', 'SLB', 'COP'],#, 'RDS.A'  
'Utilities':['NEE', 'DUK','D','AEP','SO'],
'Industrials':['GE','BA','CAT','FDX','MMM'],
'Materials':['DOW','ECL','EXP','NUE','DD'],
'Real Estate':['AMT','SPG','PLD','LEN','EQR']
            }

In [74]:
ticker_dict={'Energy':['COST'],
'Energy':['XOM', 'CVX', 'SLB', 'COP'],#, 'RDS.A'  
'Utilities':['NEE', 'DUK','D','AEP','SO'],
'Industrials':['GE','BA','CAT','FDX','MMM'],
'Materials':['DOW','ECL','EXP','NUE','DD'],
'Real Estate':['AMT','SPG','PLD','LEN','EQR']}

In [54]:
def ticker_data(tick):
    tick_url=f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={tick}&apikey=8N9R0MVWBUTQZ854'
    
    rc = requests.get(tick_url)
    json_data  = rc.json()
    
    """price_url=f'https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={tick}&apikey=F5G3ECI5K968B229'
    
    rc=requests.get(price_url)
    json_quote =rc.json()
    
       # Extract relevant data from the first JSON
    global_quote_data = json_quote.get("Global Quote", {})

    desired_keys = ["05. price", "10. change percent", "06. volume"]
    filtered_global_quote_data = {key.split(". ")[1]: global_quote_data[key] for key in desired_keys}

    # Merge the two JSONs
    merged_json = {**json_data, **filtered_global_quote_data}

    # Remove the "Global Quote" key-pair
    merged_json.pop("Global Quote", None)"""

    return merged_json

In [66]:
def get_option_data(ticker,sector):
    
    url = f"https://api.marketdata.app/v1/options/chain/{ticker}/?token=Q2FTd2tGbEhRR3FhVV9iZnhYTXd6cmNTc29nWExKcGpjRmx0OHZnZ0M0QT0"
    # Q2FTd2tGbEhRR3FhVV9iZnhYTXd6cmNTc29nWExKcGpjRmx0OHZnZ0M0QT0 
    response = requests.request("GET", url)
    
    print(response)
    data=pd.DataFrame(response.json())
    
    data=pd.DataFrame(data,columns=['optionSymbol','expiration','side','strike','bid','ask','last','openInterest','volume'])
    
    data['ticker']=ticker
    data['sector']=sector
    
    price_url = f"https://api.marketdata.app/v1/stocks/quotes/{ticker}/?token=Q2FTd2tGbEhRR3FhVV9iZnhYTXd6cmNTc29nWExKcGpjRmx0OHZnZ0M0QT0"
    #52week 
    params = {'52week': True}  # Set this to True to enable 52-week high and low data

    rq = requests.get(price_url,params)
    json_data=rq.json()

    data['lastPrice'] = json_data.get('last', [None])[0]
    data['52weekHigh'] = json_data.get('52weekHigh', [None])[0]
    data['52weekLow'] = json_data.get('52weekLow', [None])[0]
    data['tradedVolume']=json_data.get('volume', [None])[0]

    
    
    """api_request = requests.get(f'https://api.iex.cloud/v1/stock/{ticker}/logo?token=pk_f44b64891baf43279df73bfc8e58299d')
    api = json.loads(api_request.content)
    #{'url': 'https://storage.googleapis.com/iexcloud-hl37opg/api/logos/AAPL.png'}
    data['logo']=api['url']"""
    
    return data

In [5]:
def get_index_quote(index):
    url = f"https://api.marketdata.app/v1/indices/quotes/{index}/"

    data=pd.DataFrame((requests.request("GET", url)).json())
    return data['last']

In [6]:
get_index_quote('VIX')

0    12.82
Name: last, dtype: float64

In [17]:
def get_stock_quote(symbol):
    url = f"https://api.marketdata.app/v1/stocks/quotes/{symbol}/"

    data=pd.Dataframe((requests.request("GET", url)).json()) #get response data, convert to json and place data into to dataframe
    return data['last']

In [67]:
option_data=pd.DataFrame()
dfl=[]
df_tl=[]

In [69]:
for key,value in ticker_dict.items():
    for v in value:
        dfl.append(get_option_data(v,key))
        #df_tl.append(ticker_data(v))
option_data=pd.concat(dfl,ignore_index=True)

<Response [203]>
<Response [203]>
<Response [203]>
<Response [203]>
<Response [203]>
<Response [203]>
<Response [203]>
<Response [203]>
<Response [203]>
<Response [203]>
<Response [203]>
<Response [203]>
<Response [203]>
<Response [203]>
<Response [203]>
<Response [203]>
<Response [203]>
<Response [203]>
<Response [203]>
<Response [203]>
<Response [203]>
<Response [203]>
<Response [203]>
<Response [203]>
<Response [203]>
<Response [203]>
<Response [203]>
<Response [203]>
<Response [203]>
<Response [203]>
<Response [203]>
<Response [203]>
<Response [203]>
<Response [203]>
<Response [203]>
<Response [203]>
<Response [203]>
<Response [203]>
<Response [203]>
<Response [203]>
<Response [203]>
<Response [203]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [15]:
pd.DataFrame(df_tl).to_csv("ticker_data.csv", index=False)


In [71]:
option_data

,optionSymbol,expiration,side,strike,bid,ask,last,openInterest,volume,ticker,sector,lastPrice,52weekHigh,52weekLow,tradedVolume
0,MSFT231215C00160000,1702674000,call,160.0,211.85,213.50,169.45,0,0,MSFT,Technology,372.52,384.30,219.35,22882179
1,MSFT231215C00165000,1702674000,call,165.0,205.95,208.50,179.40,0,0,MSFT,Technology,372.52,384.30,219.35,22882179
2,MSFT231215C00170000,1702674000,call,170.0,200.75,203.45,202.20,0,0,MSFT,Technology,372.52,384.30,219.35,22882179
3,MSFT231215C00175000,1702674000,call,175.0,195.80,198.45,161.50,0,0,MSFT,Technology,372.52,384.30,219.35,22882179
4,MSFT231215C00180000,1702674000,call,180.0,190.80,193.55,196.93,0,0,MSFT,Technology,372.52,384.30,219.35,22882179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5633,EQR231215P00077500,1702674000,put,77.5,17.00,21.50,NaN,0,0,EQR,Real Estate,58.25,69.45,52.57,2838732
5634,EQR231215P00080000,1702674000,put,80.0,19.50,24.00,NaN,0,0,EQR,Real Estate,58.25,69.45,52.57,2838732
5635,EQR231215P00085000,1702674000,put,85.0,24.50,29.00,NaN,0,0,EQR,Real Estate,58.25,69.45,52.57,2838732
5636,EQR231215P00090000,1702674000,put,90.0,29.40,34.00,NaN,0,0,EQR,Real Estate,58.25,69.45,52.57,2838732


In [9]:
#d=option_data.loc[[option_data['volume']>5] | option_data.loc[option_data['openInterest']>5]]
d = option_data[(option_data['volume'] > 5)]# | (option_data['openInterest'] > 5)]

In [73]:
option_data.to_csv('optionChain.csv', index=True)